<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice SQL with Pandas pt. 2

_Authors: Sam Stack (DC)_

---

We've learned about relational databases and the language most use to query them, SQL.  

In this lab we are going to gain more practice converting information to a SQL DB, querying the data and then analyzing it with Python.

In [2]:
# Necessary Libraries
import pandas as pd
import sqlite3
from pandas.io import sql

#### 1.  Read in the EuroMart CSV Data.
- 'EuroMart-ListOfOrders.csv'
- 'EuroMart-OrderBreakdown.csv'
- 'EuroMart-SalesTargets.csv'

In [32]:
# Reading CSV to Dataframe
orders = pd.read_csv('./datasets/csv/EuroMart-ListOfOrders.csv', encoding = 'utf-8')
OBD =  pd.read_csv('./datasets/csv/EuroMart-OrderBreakdown.csv', encoding = 'utf-8')
sales_targets =  pd.read_csv('./datasets/csv/EuroMart-SalesTargets.csv', encoding = 'utf-8')

#### 2. Rename columns to remove any spaces.

In [33]:
# A:
for df in [orders, OBD, sales_targets]:
    names = [i.replace(' ','') for i in df.columns.tolist()]
    df.columns = names
OBD.columns

Index(['OrderID', 'ProductName', 'Discount', 'Sales', 'Profit', 'Quantity',
       'Category', 'Sub-Category'],
      dtype='object')

#### 3. Remove dollar signs from sales and profit columns in the order breakdown dataframe.

Convert the columns to float.

In [43]:
# A: 
OBD.loc[:, ['Sales', 'Profit']] = OBD[['Sales', 'Profit']].applymap(lambda x: float(str(x).replace('$', '').replace(',', '')))
OBD.head()

,OrderID,ProductName,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854.0,290.0,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,21.0,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage


#### 4. Create a SQL Database called 'EuroMart' and save the three dataframes as SQL tables. 

In [22]:
# Establishing Local DB connection
db_connection = sqlite3.connect('./datasets/sql/EuroMart.db.sqlite')


In [44]:
# A: 
lst = ['orders', 'order_breakdown', 'sales_target']
for c, df in enumerate([orders, OBD, sales_targets]):
    df.to_sql(lst[c], con = db_connection, if_exists = 'replace', index = False)

#### 5. How many orders has each Customer placed? 

In [46]:
def sql(string):
    return pd.read_sql(string, con=globals()['db_connection'])

In [56]:
# A:
sql('''
select "CustomerName", count("OrderID") as "count" from orders group by "CustomerName" order by "count" DESC
''').head()

,CustomerName,count
0,Jose Gambino,13
1,Kayla Tearle,12
2,Mark Washington,12
3,Aaron Bootman,11
4,Georgina Garner,11


In [62]:
orders.groupby('CustomerName').count()[['OrderID']].sort_values('OrderID', ascending = False).head()

,OrderID
CustomerName,
Jose Gambino,13
Mark Washington,12
Kayla Tearle,12
Aaron Bootman,11
Hayden Perkins,11


> *If you're doubting your output check using Pandas*

#### 6. Create a Query to return a table of only geographic features from the List of Orders Table.

In [66]:
# A:
sql('''
select "City", "Country", "Region", "State" from orders limit 5
''')

,City,Country,Region,State
0,Stockholm,Sweden,North,Stockholm
1,Southport,United Kingdom,North,England
2,Valence,France,Central,Auvergne-Rhône-Alpes
3,Birmingham,United Kingdom,North,England
4,Echirolles,France,Central,Auvergne-Rhône-Alpes


#### 7. Create a Query to return a table with all of the orders that had a negative profit from the Order Breakdown Table.

In [67]:
sql('''
select * from order_breakdown limit 5
''')

,OrderID,ProductName,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854.0,290.0,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,21.0,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage


In [73]:
# A:
sql('''
select * from orders
inner join
order_breakdown on orders."OrderID" = order_breakdown."OrderID"
where "Profit" < 0
''').head()

,OrderID,OrderDate,CustomerName,City,Country,Region,Segment,ShipDate,ShipMode,State,OrderID,ProductName,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,1/8/2011,Archer Hort,Toulouse,France,Central,Consumer,1/14/2011,Economy,Languedoc-Roussillon-Midi-Pyrénées,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,1/8/2011,Archer Hort,Toulouse,France,Central,Consumer,1/14/2011,Economy,Languedoc-Roussillon-Midi-Pyrénées,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


#### 8. Construct a query to return a table with the Customer Name and Product Name.  

> **Note:** This will require a join!

In [75]:
# A:
sql('''
select orders."CustomerName", order_breakdown."ProductName" from orders
inner join
order_breakdown on orders."OrderID" = order_breakdown."OrderID"
''').head()

,CustomerName,ProductName
0,Ruby Patel,"Enermax Note Cards, Premium"
1,Summer Hayward,"Dania Corner Shelving, Traditional"
2,Devin Huddleston,"Binney & Smith Sketch Pad, Easy-Erase"
3,Mary Parker,"Boston Markers, Easy-Erase"
4,Mary Parker,"Eldon Folders, Single Width"


#### 9.  How many orders for "Office Supplies" (Category) has Sweden made?

> **Note:** from this point on you'll probably be combining SQL and Pandas, in that you would use SQL querys to gather the relevant information and use Pandas to analyze it.

In [84]:
# A:
sql('''
select orders."OrderID" from orders
inner join
order_breakdown on orders."OrderID" = order_breakdown."OrderID"
where orders."Country" = 'Sweden' and order_breakdown."Category" = 'Office Supplies'
''').count()[0]

133

#### 10. What was the total sales for products that have been discounted? 

In [89]:
# A:
sql('''
select * from order_breakdown where "discount" > 0
''').Sales.sum()

1115614.0

#### 11. What is the total quantity of objects sold for each country?

In [95]:
# A:
sql('''
select orders."Country", sum(order_breakdown."Quantity") as "count" from order_breakdown
inner join orders on order_breakdown."OrderID" = orders."OrderID"
group by orders."Country"
''')

,Country,count
0,Austria,973
1,Belgium,532
2,Denmark,204
3,Finland,201
4,France,7329
5,Germany,6179
6,Ireland,392
7,Italy,3612
8,Netherlands,1526
9,Norway,261


#### 12. In what Countries are profits lowest? (Report lowest 5-10)

In [100]:
# A:
sql('''
select orders."Country", sum(order_breakdown."Profit") as "count" from order_breakdown
inner join orders on order_breakdown."OrderID" = orders."OrderID"
group by orders."Country"
order by "count" ASC
limit 2
''').iloc[0,0]

'Netherlands'

#### 13. What Counties have the best and worst Sales to Profit Ratios?
(Total Sales divided by Total Profits.)
Essentially this is saying for every dollar of product sold, how much is profit.

In [109]:
# A:
sql('''select orders."Country", sum(order_breakdown."Sales"/order_breakdown."Profit") as "ratio" from order_breakdown
inner join orders on order_breakdown."OrderID" = orders."OrderID"
group by orders."Country"
order by "ratio" DESC
''').iloc[[0,-1],:].values.tolist()

[['Germany', 9226.742840039033], ['Netherlands', -1226.4943712881009]]

#### 14. What Shipping method is most common for 'Bookcases' (Sub Category)?

In [118]:
# A:
sql('''
select orders."ShipMode", orders."OrderID" from order_breakdown 
inner join orders on orders."OrderID" = order_breakdown."OrderID"
where order_breakdown."Sub-Category" = "Bookcases"
''').ShipMode.value_counts().index[0]

'Economy'

#### 15 .What city in the Orders table generated the highest net sales?  (List all the cities and countries in descending order by net sales.)

In [125]:
# A:
sql('''
select orders."Country", orders."City", sum(order_breakdown."Sales") as "netsales" from order_breakdown 
inner join orders on orders."OrderID" = order_breakdown."OrderID"
group by orders."Country", orders."City"
order by "netsales" DESC
''').iloc[0,1]

'London'

#### BONUS: Create a Column called 'Shipping Delay' on the 'orders' table, which is the difference in days between 'Order Date' and 'Ship Date'.

In [18]:
# A:

In [19]:
# A:

#### BONUS: Update your Orders table in your Sqlite DB to include the 'Shipping Delay' feature.

In [20]:
# A:

#### BONUS: Which Product Category has the highest average 'Shipping Delay'?

In [21]:
# A:

### Challenge problem:   
**In what months and Categories were Sales Targets Exceeded?**

---

This may require a considerable amount of data processing.

In [22]:
# A: